In [1]:
from models.utils import process_batch_for_landmarks, extract_landmarks_from_batch
import torch
from torch.utils.data import DataLoader
import os
from dataset.loader import GestureDataset
import mediapipe as mp 
import torch.nn as nn 

KeyboardInterrupt: 

In [ ]:
from torchvision import transforms

file = os.path.join("D:", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", "Annot_TrainList.txt")
# frame_folders = os.path.join(".", "IPN_Hand", "frames")
hdf5_path = os.path.join("D:", "IPN_Hand", "hand_gestures.h5")
transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)
dataset = GestureDataset(hdf5_path, file, label_all_gestures=True, transform=transform)

In [ ]:
train_loader = DataLoader(dataset, batch_size=5, shuffle=True)

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

In [ ]:
for i, data in enumerate(train_loader):
    print(data[0].shape)
    processed_batch = process_batch_for_landmarks(data[0])
    landmarks = extract_landmarks_from_batch(processed_batch, hands)
    print(landmarks.shape)
    break

torch.Size([5, 60, 3, 240, 320])
(5, 60, 63)


In [ ]:
from models.LSTM import LSTMGestureClassifier

In [ ]:
model = LSTMGestureClassifier(63, 128, 2, 2)

In [ ]:
for i, data in enumerate(train_loader):
    processed_batch = process_batch_for_landmarks(data[0])
    landmarks = extract_landmarks_from_batch(processed_batch, hands)
    landmarks = torch.tensor(landmarks).float()
    print(landmarks.shape)
    out = model(landmarks)
    print(out.shape)
    break

torch.Size([5, 60, 63])
torch.Size([5, 2])


In [ ]:
weight_for_no_gesture = 0.8  # Example value
weight_for_gesture = 0.2     # Example value
weights = torch.tensor([weight_for_gesture, weight_for_no_gesture])

In [ ]:
criterion = nn.CrossEntropyLoss(weight=weights)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
device = "cpu" #  torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
for epoch in range(2):
    loss_history = []
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        processed_batch = process_batch_for_landmarks(inputs)
        landmarks = extract_landmarks_from_batch(processed_batch, hands)
        landmarks = torch.tensor(landmarks).float()

        # Forward pass
        outputs = model(landmarks)
        # outputs = outputs[:, 1]
        # labels = labels.view(-1).float()
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        print(f'Epoch [{epoch+1}], Loss: {loss.item():.4f}')

Epoch [1], Loss: 0.6907
Epoch [1], Loss: 0.4617
Epoch [1], Loss: 3.9959
Epoch [1], Loss: 1.3541
Epoch [1], Loss: 1.8308
Epoch [1], Loss: 0.4905
Epoch [1], Loss: 0.6959
Epoch [1], Loss: 0.7028
Epoch [1], Loss: 0.6815
Epoch [1], Loss: 0.8254
Epoch [1], Loss: 0.8440
Epoch [1], Loss: 0.6485
Epoch [1], Loss: 0.8749
Epoch [1], Loss: 0.6034
Epoch [1], Loss: 0.5337
Epoch [1], Loss: 0.5783
Epoch [1], Loss: 0.7619
Epoch [1], Loss: 0.4602
Epoch [1], Loss: 0.2805
Epoch [1], Loss: 1.1322
Epoch [1], Loss: 0.4554
Epoch [1], Loss: 0.6650
Epoch [1], Loss: 0.2211
Epoch [1], Loss: 0.9780
Epoch [1], Loss: 0.1491
Epoch [1], Loss: 1.0419
Epoch [1], Loss: 0.3127
Epoch [1], Loss: 0.1301
Epoch [1], Loss: 0.4283
Epoch [1], Loss: 0.8201
Epoch [1], Loss: 0.3461
Epoch [1], Loss: 0.5697
Epoch [1], Loss: 0.4075
Epoch [1], Loss: 1.6071
Epoch [1], Loss: 0.6706
Epoch [1], Loss: 0.5647
Epoch [1], Loss: 0.5605
Epoch [1], Loss: 0.1273
Epoch [1], Loss: 0.2863
Epoch [1], Loss: 1.1703
Epoch [1], Loss: 1.1576
Epoch [1], Loss:

In [ ]:
file = os.path.join("D:", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", "Annot_TestList.txt")

transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)
test_set = GestureDataset(hdf5_path, file, label_all_gestures=True, transform=transform)

In [ ]:
test_loader = DataLoader(test_set, 1, shuffle=False)

In [ ]:

for i, data in enumerate(test_loader):
    inputs, labels = data[0].to(device), data[1].to(device)
    processed_batch = process_batch_for_landmarks(inputs)
    landmarks = extract_landmarks_from_batch(processed_batch, hands)
    landmarks = torch.tensor(landmarks).float()

    outputs = model(landmarks)

    y = torch.argmax(outputs, dim=1)
    print(f"y = {y} \t gt = {labels}")

    if i == 30:
        break


y = tensor([1]) 	 gt = tensor([1])
y = tensor([1]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([1])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([1])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([1])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([1]) 	 gt = tensor([1])
y = tensor([1]) 	 gt = tensor([1])
y = tensor([0]) 	 gt = tensor([0])
y = tensor([0]) 	 gt

In [1]:
from train.train_lstm_lm_det import main
import os

In [2]:
test = os.path.join("D:", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", "Annot_TestList.txt")
train = os.path.join("D:", "IPN_Hand","annotations-20231128T085307Z-001", "annotations", "Annot_TrainList.txt")
# frame_folders = os.path.join(".", "IPN_Hand", "frames")
hdf5_path = os.path.join("D:", "IPN_Hand", "hand_gestures.h5")

In [3]:
main(hdf5_path, train, test, "lstm_det_local.pth")

{'accuracy': 0.6600000262260437, 'precision': 0.6600000262260437, 'recall': 0.6600000262260437}
